In [ ]:
#------ Prologue
import postgkyl as pg
import numpy as np
#import pylab as plt
#import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4)
import matplotlib.cm as cm
import os, fnmatch, re, sys, copy
#------ add personnal python utilities
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species
from utils import *

#------ Path towards the required folders
simdir     = 'sim_data_dir_example/2x2v_example' # This is the simulation directory relative to the g0sim
fileprefix = 'gk_bgk_im_asdex_selfOnly_2x2v_p1' # This is the prefix before the '-*.gkyl'

#----- physical quantities
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.5,                 # Elongation factor
    delta       = 0.3,                 # Triangularity factor
    q0          = None,                 # Safety factor
    x_LCFS      = 0.04,                 # position of the LCFS (= core domain width)
    x_out       = 0.08                  # SOL domain width
)

# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
simulation.add_species(ion)
simulation.add_species(elc)

simulation.set_data_param( simdir = simdir, fileprefix = fileprefix, species = simulation.species)

simulation.geom_param.load_metric(simulation.data_param.fileprefix)

#----- set up the sources
n_srcOMP=2.4e23
x_srcOMP=0.0
Te_srcOMP=2 * simulation.species['elc'].T0
Ti_srcOMP=2 * simulation.species['ion'].T0
sigma_srcOMP=0.03 * simulation.geom_param.Lx
floor_src=1e-2
def custom_density_src_profile(x,y,z):
    return n_srcOMP * (np.exp(-((x - x_srcOMP) ** 2) / (2.0 * sigma_srcOMP ** 2)) + floor_src)
def custom_temp_src_profile_elc(x, y = None, z = None):
    mask = x < (x_srcOMP + 3 * sigma_srcOMP)
    fout = np.empty_like(x)
    fout[mask] = Te_srcOMP; fout[~mask] = Te_srcOMP * 3.0 / 8.0
    return fout  
def custom_temp_src_profile_ion( x, y = None, z = None):
    mask = x < (x_srcOMP + 3 * sigma_srcOMP)
    fout = np.empty_like(x)
    fout[mask] = Ti_srcOMP; fout[~mask] = Ti_srcOMP * 3.0 / 8.0
    return fout   

simulation.set_OMPsources(n_srcOMP=n_srcOMP,x_srcOMP=x_srcOMP,
                          Te_srcOMP=Te_srcOMP,Ti_srcOMP=Ti_srcOMP,
                          sigma_srcOMP=sigma_srcOMP,floor_src=floor_src,
                          density_src_profile=custom_density_src_profile,
                          temp_src_profile_elc=custom_temp_src_profile_elc,
                          temp_src_profile_ion=custom_temp_src_profile_ion)

print("Input power:    %g MW"%(simulation.get_input_power()/1e6))
print("Input particle: %g part/s"%(simulation.get_input_particle()))

# Check available frames of the ES potential
tfs = find_available_frames(simulation,'field')

# ---- Set up normalization and units
simulation.normalize('t','mus')
simulation.normalize('x','minor radius')
simulation.normalize('y','Larmor radius')
simulation.normalize('z','pi')
simulation.normalize('fluid velocities','thermal velocity')
simulation.normalize('temperatures','eV')
simulation.normalize('pressures','Pa')
simulation.normalize('energies','MJ')

# Check available frames of the ES potential
tfs = find_available_frames(simulation,fileprefix+'-field')
print("Time frames available from %g to %g"%(tfs[0],tfs[-1]))

# ---- Show the integrated moment time evolution
time = plot_integrated_moment(simulation,fieldnames=[
    ['ne','ni'],['upare','upari'],['Tpare','Tpari'],['Tperpe','Tperpi']])

In [ ]:
#---------- 2D cut plot
#-- choose min and max time frames and cut location
cplane='xz'; ccoord = 0.0
plot_2D_cut(simulation, cplane, ccoord, 
            tf = tfs[-1], 
            fieldnames = [
               'ne','phi',
               'upare','upari',
               'Te','Ti'],
            )

In [ ]:
#------- 1D profile relaxation plot
#-- Plane location
cdirection='x'; ccoords=[0.0,0.0]
plot_1D_time_evolution(simulation,cdirection,ccoords,
                       fieldnames = ['ne','phi','upare','upari','Te','Ti','pe','pi'],
                        twindow = tfs[1:10], space_time = False,
                       )

In [ ]:
#---------------- Space-time diagrams
#-- Plane location
cdirection='x'; ccoords=[0.0,0.0]
#----------------
plot_1D_time_evolution(simulation,cdirection,ccoords,
                       fieldnames = [
                           'ne','phi',
                           'upare','upari',
                           'Te','Ti'],
                       twindow    = tfs[:], space_time = True
                       )